In [1]:
import requests
from bs4 import BeautifulSoup
import os
import re # Để sử dụng biểu thức chính quy cho phân trang


# Crawling raw HTML text

In [24]:
import requests
from bs4 import BeautifulSoup
import os
import re
import json # Vẫn giữ để nếu sau này muốn tùy chỉnh xuất riêng JSON cho PR
import uuid # Cần thiết cho ID duy nhất

# Hàm hỗ trợ để lấy số trang tối đa từ phần phân trang (không thay đổi)
def get_max_page_number(soup):
    max_page = 1
    pagination_links = soup.select('.webtong-paging #numbering a')
    current_page_elem = soup.select_one('.webtong-paging #numbering em')
    if current_page_elem:
        try:
            max_page = max(max_page, int(current_page_elem.get_text(strip=True)))
        except ValueError:
            pass
    for link in pagination_links:
        if link.has_attr('onclick'):
            match = re.search(r'submitForm\(this,\s*"list",\s*(\d+)\);', link['onclick'])
            if match:
                page_num = int(match.group(1))
                max_page = max(max_page, page_num)
    last_btn = soup.select_one('.webtong-paging .last')
    if last_btn and last_btn.has_attr('onclick'):
        match = re.search(r'submitForm\(this,\s*"list",\s*(\d+)\);', last_btn['onclick'])
        if match:
            max_page = max(max_page, int(match.group(1)))
    return max_page


def crawl_and_save_html(urls_to_crawl, output_dir="data/crawled_raw_html"):
    os.makedirs(output_dir, exist_ok=True)
    print(f"Thư mục '{output_dir}' đã sẵn sàng để lưu trữ HTML.")

    for name, base_url in urls_to_crawl.items():
        print(f"\n--- Đang xử lý: {name} từ {base_url} ---")
        
        # Đường dẫn cho file HTML tổng hợp hoặc file HTML đơn
        file_path_to_save = os.path.join(output_dir, f"{name}.html")
        if name == "Press_Release":
            file_path_to_save = os.path.join(output_dir, f"{name}_combined.html")

        if os.path.exists(file_path_to_save):
            print(f"  File '{file_path_to_save}' đã tồn tại. Bỏ qua crawl cho {name}.")
            continue # Bỏ qua nếu file đã tồn tại

        # Xử lý riêng cho Press Release (có phân trang)
        if name == "Press_Release":
            with open(file_path_to_save, "w", encoding="utf-8") as combined_file:
                # Bọc toàn bộ nội dung trong một thẻ <main> để UnstructuredHTMLLoader dễ nhận diện nội dung chính
                combined_file.write("<!DOCTYPE html>\n<html><head><meta charset='utf-8'></head><body>\n")
                combined_file.write("<main>\n") # Thẻ <main> để bọc nội dung chính
                
                current_page = 1
                max_page_found = 1 # Ban đầu giả định chỉ có 1 trang

                while current_page <= max_page_found:
                    url = f"{base_url}&pageNum={current_page}" if current_page > 1 else base_url
                    print(f"  > Đang tải trang {current_page} của {name} từ {url}...")
                    try:
                        response = requests.get(url, timeout=20)
                        response.raise_for_status()
                        html_content = response.text
                        soup = BeautifulSoup(html_content, 'html.parser')

                        # Cập nhật số trang tối đa (để biết khi nào dừng)
                        new_max_page = get_max_page_number(soup)
                        if new_max_page > max_page_found:
                            max_page_found = new_max_page
                            print(f"  > Cập nhật tổng số trang cho {name} thành: {max_page_found}")

                        # Tìm phần chứa các bài viết (từng thẻ <li> của mỗi bài)
                        article_list_items = soup.select('.board_list1 .event > li')
                        
                        if article_list_items:
                            for item in article_list_items:
                                # TRỌNG TÂM ĐIỀU CHỈNH: Bọc mỗi <li> trong thẻ <article>
                                # UnstructuredHTMLLoader rất giỏi nhận diện <article> như một tài liệu độc lập
                                combined_file.write(f"<article data-source-url='{url}' data-article-id='{str(uuid.uuid4())}'>\n")
                                combined_file.write(str(item) + "\n")
                                combined_file.write("</article>\n")
                            print(f"  + Đã thêm {len(article_list_items)} bài viết từ trang {current_page} (đã bọc <article>) vào file tổng.")
                        else:
                            print(f"  ! Không tìm thấy bài viết nào trên trang {current_page}. Dừng crawl {name}.")
                            break

                        current_page += 1

                        # Điều kiện dừng vòng lặp
                        if current_page > max_page_found:
                            break

                    except requests.exceptions.RequestException as e:
                        print(f"  ! Lỗi khi tải trang {url}: {e}. Dừng crawl {name}.")
                        break
                    except Exception as e:
                        print(f"  ! Lỗi xử lý trang {url}: {e}. Dừng crawl {name}.")
                        break

                combined_file.write("</main>\n") # Đóng thẻ <main>
                combined_file.write("</body></html>\n")
                print(f"✅ Đã lưu tất cả nội dung Press Release vào: {file_path_to_save}")

        else: # Xử lý các trang không phân trang (HTML đơn)
            try:
                response = requests.get(base_url, timeout=20)
                response.raise_for_status()
                html_content = response.text

                with open(file_path_to_save, "w", encoding="utf-8") as f:
                    f.write(html_content)
                print(f"  Đã lưu: {file_path_to_save}")

            except requests.exceptions.RequestException as e:
                print(f"  ! Lỗi khi tải trang {base_url}: {e}")
            except Exception as e:
                print(f"  ! Lỗi xử lý trang {base_url}: {e}")

    print("\n--- ✅ Hoàn tất quá trình crawl và lưu HTML ---")


if __name__ == "__main__":
    urls_and_names = {
        "Information_of_Apec": "https://apec2025.kr/?menuno=89",
        "Introduction_About_Apec_Korea_2025": "https://apec2025.kr/?menuno=91",
        "Emblem_and_Theme": "https://apec2025.kr/?menuno=92",
        "Meetings": "https://apec2025.kr/?menuno=93",
        "Side_Events": "https://apec2025.kr/?menuno=94",
        "Documents_HRDDM": "https://apec2025.kr/?menuno=148",
        "Documents_AEMM": "http://apec2025.kr/?menuno=149", 
        "Documents_MRT": "https://apec2025.kr/?menuno=150",
        "Notices": "https://apec2025.kr/?menuno=15",
        "Press_Release": "https://apec2025.kr/?menuno=16", # Trang này có phân trang
        "Korea_in_Brief": "https://apec2025.kr/?menuno=18",
        "Practical_Information": "https://apec2025.kr/?menuno=22",
        "About_Gyeongju": "https://apec2025.kr/?menuno=102",
        "Transportation_of_Gyeongju": "https://apec2025.kr/?menuno=137",
        "Heritage_Gyeongju": "https://apec2025.kr/?menuno=108",
        "Attraction_of_Gyeongju": "https://apec2025.kr/?menuno=138",
        "About_Jeju": "https://apec2025.kr/?menuno=103",
        "Transportation_Jeju": "https://apec2025.kr/?menuno=141",
        "Nature_Culture_Jeju": "https://apec2025.kr/?menuno=114",
        "Themed_Travel_Jeju": "https://apec2025.kr/?menuno=115",
        "About_Incheon": "https://apec2025.kr/?menuno=104",
        "Attractions_Incheon": "https://apec2025.kr/?menuno=117",
        "Local_Eateries_Incheon": "https://apec2025.kr/?menuno=118",
        "About_Busan": "https://apec2025.kr/?menuno=106",
        "About_Seoul": "https://apec2025.kr/?menuno=24",
    }
    
    crawl_and_save_html(urls_and_names)

Thư mục 'data/crawled_raw_html' đã sẵn sàng để lưu trữ HTML.

--- Đang xử lý: Information_of_Apec từ https://apec2025.kr/?menuno=89 ---
  Đã lưu: data/crawled_raw_html\Information_of_Apec.html

--- Đang xử lý: Introduction_About_Apec_Korea_2025 từ https://apec2025.kr/?menuno=91 ---
  Đã lưu: data/crawled_raw_html\Introduction_About_Apec_Korea_2025.html

--- Đang xử lý: Emblem_and_Theme từ https://apec2025.kr/?menuno=92 ---
  Đã lưu: data/crawled_raw_html\Emblem_and_Theme.html

--- Đang xử lý: Meetings từ https://apec2025.kr/?menuno=93 ---
  Đã lưu: data/crawled_raw_html\Meetings.html

--- Đang xử lý: Side_Events từ https://apec2025.kr/?menuno=94 ---
  Đã lưu: data/crawled_raw_html\Side_Events.html

--- Đang xử lý: Documents_HRDDM từ https://apec2025.kr/?menuno=148 ---
  Đã lưu: data/crawled_raw_html\Documents_HRDDM.html

--- Đang xử lý: Documents_AEMM từ http://apec2025.kr/?menuno=149 ---
  Đã lưu: data/crawled_raw_html\Documents_AEMM.html

--- Đang xử lý: Documents_MRT từ https://apec

# Preprocessing

In [ ]:
!pip install unstructured

In [25]:
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import os
import json
import glob
import uuid
from bs4 import BeautifulSoup

In [30]:
def extract_main_content_html(html_file_path):
    """
    Extracts the HTML content from the main content area of the page.
    Assumes main content is within <div id="contents">...</div>.
    """
    try:
        with open(html_file_path, 'r', encoding='utf-8') as f:
            html_content = f.read()
    except FileNotFoundError:
        print(f"Lỗi: File HTML không tìm thấy tại '{html_file_path}'")
        return None

    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Tìm thẻ div có id="contents"
    main_contents_div = soup.find('div', id='contents')
    
    if main_contents_div:
        # Loại bỏ các thẻ không liên quan đến nội dung chính bên trong main_contents_div
        # Ví dụ: scripts, styles, các menu con trong sidebar nếu có, pagination, v.v.
        for tag in main_contents_div(['script', 'style', 'nav', 'form', 'img', 'svg', 'header', 'footer']):
            tag.decompose()
        
        # Trả về HTML đã được làm sạch của khu vực nội dung chính
        return str(main_contents_div)
    else:
        print(f"Cảnh báo: Không tìm thấy div #contents trong file {html_file_path}. Sẽ xử lý toàn bộ HTML.")
        return html_content # Trả về toàn bộ HTML nếu không tìm thấy khu vực chính

def process_html_files_to_chunks_smartly(html_dir="backend/data/crawled_raw_html", output_json_path="backend/data/apec_all_chunks.json"):
    all_chunks_data = []
    
    output_data_dir = os.path.dirname(output_json_path)
    os.makedirs(output_data_dir, exist_ok=True)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        add_start_index=True
    )

    html_files = glob.glob(os.path.join(html_dir, "*.html"))
    if not html_files:
        print(f"Không tìm thấy file HTML nào trong thư mục '{html_dir}'. Vui lòng crawl dữ liệu trước.")
        return

    print(f"Tìm thấy {len(html_files)} file HTML để xử lý.")

    for html_file in html_files:
        file_name = os.path.basename(html_file)
        print(f"Đang xử lý file: {file_name}")

        try:
            # Bước mới: Chỉ lấy phần HTML của khu vực nội dung chính
            main_content_html = extract_main_content_html(html_file)
            
            if main_content_html:
                # Tạo một file tạm thời chỉ chứa nội dung chính để UnstructuredHTMLLoader đọc
                temp_html_path = f"{html_file}.temp.html"
                with open(temp_html_path, "w", encoding="utf-8") as temp_f:
                    temp_f.write(main_content_html)

                loader = UnstructuredHTMLLoader(temp_html_path)
                documents = loader.load() 
                
                # Xóa file tạm sau khi đã load
                os.remove(temp_html_path)

                chunks = text_splitter.split_documents(documents)

                for chunk in chunks:
                    topic_from_filename = file_name.replace(".html", "").replace("_page_", " Page ").replace("_", " ")
                    
                    # Cố gắng lấy topic/sub_topic từ metadata của UnstructuredHTMLLoader nếu có
                    # Unstructured loader tự tạo metadata 'url' từ path file tạm, nên sẽ không phải url gốc
                    # Bạn có thể tự ánh xạ lại url gốc nếu cần
                    
                    # Kiểm tra và làm sạch content một lần nữa nếu cần
                    cleaned_content = ' '.join(chunk.page_content.split()) 
                    
                    chunk_data = {
                        "id": str(uuid.uuid4()),
                        "topic": chunk.metadata.get("category", topic_from_filename), 
                        "sub_topic": chunk.metadata.get("title", chunk.metadata.get("header", "N/A")), 
                        "content": cleaned_content, # Sử dụng nội dung đã làm sạch
                        "source_file": file_name,
                        # "source_url": "URL_GOC_CUA_TRANG_NAY" # Bạn cần một cách để ánh xạ filename về URL gốc
                    }
                    all_chunks_data.append(chunk_data)
            else:
                print(f"Không tìm thấy nội dung chính để xử lý từ file: {file_name}. Bỏ qua file này.")

        except Exception as e:
            print(f"Lỗi khi xử lý file '{html_file}': {e}")
            
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(all_chunks_data, f, ensure_ascii=False, indent=4)

    print(f"\n--- Đã xử lý {len(html_files)} file HTML và lưu {len(all_chunks_data)} chunks vào '{output_json_path}' ---")
    
    if all_chunks_data:
        print("\n--- Chunk mẫu đầu tiên sau khi cải thiện ---")
        print(json.dumps(all_chunks_data[0], ensure_ascii=False, indent=4))

# Để chạy:
if __name__ == "__main__":
    # Đảm bảo thư mục 'crawled_html' tồn tại và chứa các file HTML
    # Bạn có thể chạy script crawl_all_html.py trước
    # Vị trí của script này: backend/scripts/process_html_to_chunks_smartly.py
    # Thư mục chứa HTML đã crawl: crawled_html (nếu script crawl_all_html.py đặt ngang cấp)
    # Hoặc backend/crawled_html (nếu bạn muốn tổ chức bên trong backend)

    # Giả sử bạn đang chạy script từ thư mục gốc của dự án
    # và thư mục 'crawled_html' nằm ở đó
    html_input_directory = "data/crawled_raw_html"
    output_json_file = "data/json_chunks/apec_all_chunks.json"

    process_html_files_to_chunks_smartly(html_input_directory, output_json_file)

Tìm thấy 25 file HTML để xử lý.
Đang xử lý file: About_Busan.html
Đang xử lý file: About_Gyeongju.html
Đang xử lý file: About_Incheon.html
Đang xử lý file: About_Jeju.html
Đang xử lý file: About_Seoul.html
Đang xử lý file: Attractions_Incheon.html
Đang xử lý file: Attraction_of_Gyeongju.html
Đang xử lý file: Documents_AEMM.html
Đang xử lý file: Documents_HRDDM.html
Đang xử lý file: Documents_MRT.html
Đang xử lý file: Emblem_and_Theme.html
Đang xử lý file: Heritage_Gyeongju.html
Đang xử lý file: Information_of_Apec.html
Đang xử lý file: Introduction_About_Apec_Korea_2025.html


short text: "Notices". Defaulting to English.


Đang xử lý file: Korea_in_Brief.html
Đang xử lý file: Local_Eateries_Incheon.html
Đang xử lý file: Meetings.html
Đang xử lý file: Nature_Culture_Jeju.html
Đang xử lý file: Notices.html
Đang xử lý file: Practical_Information.html
Đang xử lý file: Press_Release_combined.html
Cảnh báo: Không tìm thấy div #contents trong file data/crawled_raw_html\Press_Release_combined.html. Sẽ xử lý toàn bộ HTML.
Đang xử lý file: Side_Events.html
Đang xử lý file: Themed_Travel_Jeju.html
Đang xử lý file: Transportation_Jeju.html
Đang xử lý file: Transportation_of_Gyeongju.html

--- Đã xử lý 25 file HTML và lưu 261 chunks vào 'data/json_chunks/apec_all_chunks.json' ---

--- Chunk mẫu đầu tiên sau khi cải thiện ---
{
    "id": "f1e9e541-6d4c-4b6f-8bd8-b5d617273523",
    "topic": "About Busan",
    "sub_topic": "N/A",
    "content": "Busan About Busan About Busan",
    "source_file": "About_Busan.html"
}
